In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('E:/git_exercise/Machine learning/boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [5]:
# DataFrame 'data'
# целевая переменная'MEDV'

# Разделение выборки на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(data.drop('MEDV', axis=1), data['MEDV'], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test

(         CRIM    ZN  INDUS  CHAS     NOX     RM   AGE     DIS   RAD    TAX  \
 477  15.02340   0.0  18.10   0.0  0.6140  5.304  97.3  2.1007  24.0  666.0   
 15    0.62739   0.0   8.14   0.0  0.5380  5.834  56.5  4.4986   4.0  307.0   
 332   0.03466  35.0   6.06   0.0  0.4379  6.031  23.3  6.6407   1.0  304.0   
 423   7.05042   0.0  18.10   0.0  0.6140  6.103  85.1  2.0218  24.0  666.0   
 19    0.72580   0.0   8.14   0.0  0.5380  5.727  69.5  3.7965   4.0  307.0   
 ..        ...   ...    ...   ...     ...    ...   ...     ...   ...    ...   
 106   0.17120   0.0   8.56   0.0  0.5200  5.836  91.9  2.2110   5.0  384.0   
 270   0.29916  20.0   6.96   0.0  0.4640  5.856  42.1  4.4290   3.0  223.0   
 348   0.01501  80.0   2.01   0.0  0.4350  6.635  29.7  8.3440   4.0  280.0   
 435  11.16040   0.0  18.10   0.0  0.7400  6.629  94.6  2.1247  24.0  666.0   
 102   0.22876   0.0   8.56   0.0  0.5200  6.405  85.4  2.7147   5.0  384.0   
 
      PTRATIO       B  LSTAT  
 477     20.2  349.

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
lr = LinearRegression()
lr.fit(X_train, y_train)

ridge = Ridge()
ridge.fit(X_train, y_train)

lasso = Lasso()
lasso.fit(X_train, y_train)

lr_r2 = r2_score(y_test, lr.predict(X_test))
ridge_r2 = r2_score(y_test, ridge.predict(X_test))
lasso_r2 = r2_score(y_test, lasso.predict(X_test))

print("R2 для стандартной регрессии:", lr_r2)
print("R2 для Ridge:", ridge_r2)
print("R2 для Lasso:", lasso_r2)

R2 для стандартной регрессии: 0.6684825753971619
R2 для Ridge: 0.6659608075261688
R2 для Lasso: 0.6668687223368214


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:
# Создаем модель Ridge
ridge_model = Ridge()

# Создаем модель Lasso
lasso_model = Lasso()


border_values=[0.0001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000]
# Создаем словарь параметров для поиска
params = {'alpha': border_values}

# Создаем экземпляр GridSearchCV для модели Ridge
ridge_grid_search = GridSearchCV(ridge_model, param_grid=params)

# Создаем экземпляр GridSearchCV для модели Lasso
lasso_grid_search = GridSearchCV(lasso_model, param_grid=params)

# Создаем модель RidgeCV
ridge_cv = RidgeCV(alphas=border_values)

# Создаем модель LassoCV
lasso_cv = LassoCV(alphas=border_values)

# Запускаем поиск
ridge_grid_search.fit(X_train, y_train)
lasso_grid_search.fit(X_train, y_train)
ridge_cv.fit(X_train, y_train)
lasso_cv.fit(X_train, y_train)

# Выводим результаты
print("Лучший параметр для Ridge: ", ridge_grid_search.best_params_['alpha'])
print("Лучший параметр для Lasso: ", lasso_grid_search.best_params_['alpha'])
print("Лучший параметр для RidgeCV: ", ridge_cv.alpha_)
print("Лучший параметр для LassoCV: ", lasso_cv.alpha_)

Лучший параметр для Ridge:  0.0001
Лучший параметр для Lasso:  0.0001
Лучший параметр для RidgeCV:  0.01
Лучший параметр для LassoCV:  0.0001


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:
# Создаем модель Ridge
#ridge_model = Ridge()

# Создаем модель Lasso
#lasso_model = Lasso()

# Создаем Pipeline с StandardScaler
scaler = StandardScaler()
pipeline_ridge = Pipeline([('scaler', scaler), ('ridge', ridge_model)])

# Создаем Pipeline с MinMaxScaler
scaler = MinMaxScaler()
pipeline_lasso = Pipeline([('scaler', scaler), ('lasso', lasso_model)])

# Кросс-валидация
ridge_scores = cross_val_score(pipeline_ridge, X_train, y_train, scoring='r2')
lasso_scores = cross_val_score(pipeline_lasso, X_train, y_train, scoring='r2')


print("Средний R2 для Ridge: ", ridge_scores.mean())
print("Средний R2 для Lasso: ", lasso_scores.mean())

Средний R2 для Ridge:  0.7245783031128651
Средний R2 для Lasso:  0.2458342814681857


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [8]:
scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

ridge_cv = RidgeCV()
lasso_cv = LassoCV()

pipeline_ridge = Pipeline([('scaler', scaler), ('ridge_cv', ridge_cv)])
pipeline_lasso = Pipeline([('scaler', minmax_scaler), ('lasso_cv', lasso_cv)])

pipeline_ridge.fit(X_train, y_train)
pipeline_lasso.fit(X_train, y_train)

best_alpha_ridge = pipeline_ridge.named_steps['ridge_cv'].alpha_
best_alpha_lasso = pipeline_lasso.named_steps['lasso_cv'].alpha_

ridge_model = Ridge(alpha=best_alpha_ridge)
ridge_model.fit(X_train, y_train)

lasso_model = Lasso(alpha=best_alpha_lasso)
lasso_model.fit(X_train, y_train)

r2_ridge = r2_score(y_test, ridge_model.predict(X_test))
r2_lasso = r2_score(y_test, lasso_model.predict(X_test))

r2_ridge, r2_lasso

(0.6636035715018218, 0.6685249100083176)

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [9]:
ridge_model = Ridge()
lasso_model = Lasso()

stand_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

border_values=[0.0001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000]

params = {'ridge__alpha': [10 ** x for x in range(-5, 5)]}


poly_features = PolynomialFeatures(2)



stand_ridge = Pipeline([('poly_features', poly_features),('stand', stand_scaler), ('ridge', ridge_model)])
grid_search = GridSearchCV(stand_ridge, param_grid=params, cv=3).fit(X_train, y_train)
print('best alpha for standard scaling:', 
      grid_search.best_estimator_.get_params()['ridge__alpha'])
print(grid_search.best_score_)
print('score:', grid_search.score(X_test, y_test))

minmax_ridge = Pipeline([('poly_features', poly_features),('minmax', minmax_scaler), ('ridge', ridge_model)])
grid_search = GridSearchCV(minmax_ridge, param_grid=params, cv=3).fit(X_train, y_train)
print('best alpha for minmax scaling:', 
      grid_search.best_estimator_.get_params()['ridge__alpha'])
print(grid_search.best_score_)
print('score:', grid_search.score(X_test, y_test))

best alpha for standard scaling: 1
0.8358387061836421
score: 0.8473703650202432
best alpha for minmax scaling: 0.1
0.8458593856595996
score: 0.8518360877299457


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [10]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score


# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создаем полиномиальные признаки
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Создаем модели Ridge и Lasso с различными значениями альфа
ridge_models = [Ridge(alpha=a) for a in [0.001, 0.01, 0.1, 1, 10, 100]]
lasso_models = [Lasso(alpha=a) for a in [0.001, 0.01, 0.1, 1, 10, 100]]

# Обучаем модели
ridge_models_poly = [make_pipeline(poly, ridge_model) for ridge_model in ridge_models]
lasso_models_poly = [make_pipeline(poly, lasso_model) for lasso_model in lasso_models]

for ridge_model_poly, lasso_model_poly in zip(ridge_models_poly, lasso_models_poly):
    ridge_model_poly.fit(X_train_poly, y_train)
    lasso_model_poly.fit(X_train_poly, y_train)

    # Вычисляем R2 для каждой модели
    ridge_r2 = r2_score(y_test, ridge_model_poly.predict(X_test_poly))
    lasso_r2 = r2_score(y_test, lasso_model_poly.predict(X_test_poly))

    print(f"Ridge (alpha={ridge_model_poly.get_params()['ridge__alpha']}): R2 = {ridge_r2:.4f}")
    print(f"Lasso (alpha={lasso_model_poly.get_params()['lasso__alpha']}): R2 = {lasso_r2:.4f}")

E:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.859e+02, tolerance: 3.510e+00
  model = cd_fast.enet_coordinate_descent(


Ridge (alpha=0.001): R2 = -2.4244
Lasso (alpha=0.001): R2 = 0.4792


E:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.689e+02, tolerance: 3.510e+00
  model = cd_fast.enet_coordinate_descent(


Ridge (alpha=0.01): R2 = -1.9826
Lasso (alpha=0.01): R2 = 0.7352


E:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.657e+02, tolerance: 3.510e+00
  model = cd_fast.enet_coordinate_descent(


Ridge (alpha=0.1): R2 = -0.3707
Lasso (alpha=0.1): R2 = 0.6383
Ridge (alpha=1): R2 = -0.7205
Lasso (alpha=1): R2 = 0.6621
Ridge (alpha=10): R2 = 0.6019
Lasso (alpha=10): R2 = 0.3878
Ridge (alpha=100): R2 = 0.7878
Lasso (alpha=100): R2 = -0.0159


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [13]:
# Создаем функцию, которая возвращает Pipeline с различными параметрами
def create_pipeline(reg_type, reg_coef, poly_degree):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=poly_degree)),
        ('reg', reg_type(alpha=reg_coef))
    ])
    return pipeline

# Создаем список параметров для GridSearchCV
param_grid = {
    'reg__alpha': [0.1, 0.5, 1.0],
    'poly__degree': [2, 3, 4]
}

# Создаем модель и запускаем GridSearchCV
model = GridSearchCV(create_pipeline(Lasso, 1.0, 2), param_grid)
model.fit(X_train, y_train)

# Выводим параметры лучшей модели
best_params = model.best_params_
print('Параметры лучшей модели:\n', best_params)

# Вычисляем R2 для лучшей модели
y_pred = model.predict(X_test)
r2_best_model = r2_score(y_test, y_pred)

# Записываем результаты в DataFrame
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]
print(results_regression.loc[23])

E:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.090e+02, tolerance: 2.882e+00
  model = cd_fast.enet_coordinate_descent(
E:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.725e+01, tolerance: 2.764e+00
  model = cd_fast.enet_coordinate_descent(
E:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.995e+02, tolerance: 2.710e+00
  model = cd_fast.enet_

Параметры лучшей модели:
 {'poly__degree': 2, 'reg__alpha': 0.1}
model    Best_Model
task          task8
R2         0.812814
Name: 23, dtype: object


E:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.986e+01, tolerance: 2.948e+00
  model = cd_fast.enet_coordinate_descent(


In [ ]:
results_regression

http://archive.ics.uci.edu/ml/datasets/Adult

In [7]:
data2 = pd.read_csv('E:/git_exercise/Machine learning/adult.csv')
data2.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [38]:
### Ваш код ###

data2['class'] = data2['class'].replace(['<=50K', '>50K'], [1, 0])

# разделить DataFrame на признаки и целевую переменную
X = data2.drop('class', axis=1)
y = data2['class']

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [26]:
### Ваш код ###
'''
f1_most_frequent = 0
acc_most_frequent = 0
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]
'''
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Столбец, который содержит целевую переменную
y_true = data2['class']

# Самый часто встречающийся класс
most_common_class = y_true.value_counts().index[0]

# Создание предсказаний, присвоив всем значениям 'most_common_class'
y_pred = y_true.apply(lambda x: most_common_class if x == most_common_class else x)

# Вычисление метрик
accuracy = accuracy_score(y_true, y_pred)
f1_score = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy}')
print(f'F1 score: {f1_score}')

Accuracy: 1.0
F1 score: 1.0


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [9]:
### Ваш код ###
from sklearn.impute import SimpleImputer
missing_values = data2.isnull().sum()
imputer = SimpleImputer(strategy='most_frequent')
data2 = pd.DataFrame(imputer.fit_transform(data2), columns=data2.columns)
data2

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,1
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,1
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,1
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,1


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [33]:
data = pd.read_csv('E:/git_exercise/Machine learning/adult.csv')
# колонки с числовыми переменными
numeric_columns = data.select_dtypes(include=['number']).columns

# колонки с категориальными переменными
categorical_columns = data.select_dtypes(include=['category', 'object']).columns

# оба списка в один
all_columns = numeric_columns.union(categorical_columns)

print('Числовые: ', numeric_columns)
print('Категориальные: ', categorical_columns)
print('Все колонки: ', all_columns)

Числовые:  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')
Категориальные:  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'class'],
      dtype='object')
Все колонки:  Index(['age', 'capital-gain', 'capital-loss', 'class', 'education',
       'education-num', 'fnlwgt', 'hours-per-week', 'marital-status',
       'native-country', 'occupation', 'race', 'relationship', 'sex',
       'workclass'],
      dtype='object')


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [122]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('E:/git_exercise/Machine learning/adult.csv')
df['class'] = df['class'].replace(['<=50K', '>50K'], [1, 0])

X = df.drop('class', axis=1)
y = df['class']

categorical_columns =X.select_dtypes(include=['category', 'object']).columns
numeric_columns = X.select_dtypes(include=['number']).columns

# Создать препроцессоры для каждой колонки
numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(sparse_output=False)

# Создать ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Создать pipeline, который будет использовать препроцессор и модель LogisticRegression
logistic_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression())])
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())])
linear_svc_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LinearSVC())])

# Применяем cross_val_score к каждому pipeline
logistic_scores = cross_val_score(logistic_pipeline, X, y, cv=5, scoring='accuracy')
f1_log_scores = cross_val_score(logistic_pipeline, X, y, cv=5, scoring='f1')
knn_scores = cross_val_score(knn_pipeline, X, y, cv=5, scoring='accuracy')
f1_knn_scores=cross_val_score(knn_pipeline, X, y, cv=5, scoring='f1')
linear_svc_scores = cross_val_score(linear_svc_pipeline, X, y, cv=5, scoring='accuracy')
f1_lin_scores = cross_val_score(linear_svc_pipeline, X, y, cv=5, scoring='f1')

print(logistic_scores, f1_log_scores, knn_scores, f1_knn_scores, linear_svc_scores, f1_lin_scores)

E:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

[0.84901218 0.84860272        nan 0.85145373 0.8499181 ] [0.90367661 0.90327644        nan 0.90505791 0.90417048] [0.82505886 0.8246494         nan 0.82411957 0.82350532] [0.88745473 0.88662387        nan 0.88664555 0.88608431] [0.85065002 0.85167366        nan 0.85176085 0.8531941 ] [0.90496971 0.90534979        nan 0.90553236 0.90639687]


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [127]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('E:/git_exercise/Machine learning/adult.csv')
df['class'] = df['class'].replace(['<=50K', '>50K'], [1, 0])
imputer = SimpleImputer(missing_values='?',strategy='most_frequent')
imputer.fit_transform(df)

X = df.drop('class', axis=1)
y = df['class']


categorical_columns =X.select_dtypes(include=['category', 'object']).columns
numeric_columns = X.select_dtypes(include=['number']).columns

# Создать препроцессоры для каждой колонки
numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(sparse_output=False)

# Создать ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Создать pipeline, который будет использовать препроцессор и модель LogisticRegression
logistic_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression())])
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())])
linear_svc_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LinearSVC())])

# Применяем cross_val_score к каждому pipeline
logistic_scores = cross_val_score(logistic_pipeline, X, y, cv=5, scoring='accuracy')
f1_log_scores = cross_val_score(logistic_pipeline, X, y, cv=5, scoring='f1')
knn_scores = cross_val_score(knn_pipeline, X, y, cv=5, scoring='accuracy')
f1_knn_scores=cross_val_score(knn_pipeline, X, y, cv=5, scoring='f1')
linear_svc_scores = cross_val_score(linear_svc_pipeline, X, y, cv=5, scoring='accuracy')
f1_lin_scores = cross_val_score(linear_svc_pipeline, X, y, cv=5, scoring='f1')

print(logistic_scores, f1_log_scores, knn_scores, f1_knn_scores, linear_svc_scores, f1_lin_scores)

E:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

[0.84901218 0.84860272        nan 0.85145373 0.8499181 ] [0.90367661 0.90327644        nan 0.90505791 0.90417048] [0.82505886 0.8246494         nan 0.82411957 0.82350532] [0.88745473 0.88662387        nan 0.88664555 0.88608431] [0.85065002 0.85167366        nan 0.85176085 0.8531941 ] [0.90496971 0.90534979        nan 0.90553236 0.90639687]


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [130]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

### Ваш код ###


df = pd.read_csv('E:/git_exercise/Machine learning/adult.csv')
df['class'] = df['class'].replace(['<=50K', '>50K'], [1, 0])
df = df.replace('?', np.nan)
df = df.dropna()


X = df.drop('class', axis=1)
y = df['class']


categorical_columns =X.select_dtypes(include=['category', 'object']).columns
numeric_columns = X.select_dtypes(include=['number']).columns

# Создать препроцессоры для каждой колонки
numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(sparse_output=False)

# Создать ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Создать pipeline, который будет использовать препроцессор и модель LogisticRegression
logistic_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression())])
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())])
linear_svc_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LinearSVC())])

# Применяем cross_val_score к каждому pipeline
logistic_scores = cross_val_score(logistic_pipeline, X, y, cv=5, scoring='accuracy')
f1_log_scores = cross_val_score(logistic_pipeline, X, y, cv=5, scoring='f1')
knn_scores = cross_val_score(knn_pipeline, X, y, cv=5, scoring='accuracy')
f1_knn_scores=cross_val_score(knn_pipeline, X, y, cv=5, scoring='f1')
linear_svc_scores = cross_val_score(linear_svc_pipeline, X, y, cv=5, scoring='accuracy')
f1_lin_scores = cross_val_score(linear_svc_pipeline, X, y, cv=5, scoring='f1')

print(logistic_scores, f1_log_scores, knn_scores, f1_knn_scores, linear_svc_scores, f1_lin_scores)

E:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

[0.84400221 0.84433389        nan 0.84575409 0.84586466] [0.89925027 0.89944294        nan 0.90032154 0.90049964] [0.82056385 0.82144831        nan 0.8194383  0.81711632] [0.88317858 0.88328395        nan 0.88223841 0.88057762] [0.84610282 0.84754008        nan 0.84730208 0.84763379] [0.90092527 0.90166156        nan 0.90151893 0.90171184]


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score

df = pd.read_csv('E:/git_exercise/Machine learning/adult.csv')
df['class'] = df['class'].replace(['<=50K', '>50K'], [1, 0])

imputer = SimpleImputer(missing_values='?',strategy='most_frequent')
imputer.fit_transform(df)

X = df.drop('class', axis=1)
y = df['class']

categorical_columns =X.select_dtypes(include=['category', 'object']).columns
numeric_columns = X.select_dtypes(include=['number']).columns

numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

model_rf = RandomForestClassifier()
model_gb = GradientBoostingClassifier()

X_transformed = preprocessor.fit_transform(X)

score_rf = cross_val_score(model_rf, X_transformed, y, cv=5, scoring='accuracy')
score_gb = cross_val_score(model_gb, X_transformed, y, cv=5, scoring='accuracy')

print("Средний результат для RandomForestClassifier:", score_rf.mean())
print("Средний результат для GradientBoostingClassifier:", score_gb.mean())

Средний результат для RandomForestClassifier: 0.851562181813487
Средний результат для GradientBoostingClassifier: 0.8675731196536806


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [134]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score


df = pd.read_csv('E:/git_exercise/Machine learning/adult.csv')
df['class'] = df['class'].replace(['<=50K', '>50K'], [1, 0])

imputer = SimpleImputer(missing_values='?',strategy='most_frequent')
imputer.fit_transform(df)

X = df.drop('class', axis=1)
y = df['class']

categorical_columns =X.select_dtypes(include=['category', 'object']).columns
numeric_columns = X.select_dtypes(include=['number']).columns

numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])



X_transformed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Масштабирование признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Кодирование признаков
#from sklearn.preprocessing import OneHotEncoder
#encoder = OneHotEncoder()
#X_train = encoder.fit_transform(X_train).toarray()
#X_test = encoder.transform(X_test).toarray()

# Заполнение пропусков
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Подбор наилучшей модели
models = [
    ('RandomForestClassifier', RandomForestClassifier()),
    ('GradientBoostingClassifier', GradientBoostingClassifier()),
    ('LogisticRegression', LogisticRegression()),
    ('KNeighborsClassifier', KNeighborsClassifier()),
    ('LinearSVC', LinearSVC())
]

# Обучение моделей
for name, model in models:
    model.fit(X_train, y_train)

# Прогнозирование на тестовой выборке
predictions = [model.predict(X_test) for name, model in models]

# Вычисление метрик
scores = [
    (name, accuracy_score(y_test, prediction), f1_score(y_test, prediction))
    for name, model in models
    for prediction in predictions
]

# Вывод результатов
for name, accuracy, f1 in scores:
    print(f'{name}: accuracy={accuracy:.4f}, f1_score={f1:.4f}')
    


E:\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
E:\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomForestClassifier: accuracy=0.8515, f1_score=0.9044
RandomForestClassifier: accuracy=0.8667, f1_score=0.9156
RandomForestClassifier: accuracy=0.8511, f1_score=0.9050
RandomForestClassifier: accuracy=0.8195, f1_score=0.8838
RandomForestClassifier: accuracy=0.8504, f1_score=0.9047
GradientBoostingClassifier: accuracy=0.8515, f1_score=0.9044
GradientBoostingClassifier: accuracy=0.8667, f1_score=0.9156
GradientBoostingClassifier: accuracy=0.8511, f1_score=0.9050
GradientBoostingClassifier: accuracy=0.8195, f1_score=0.8838
GradientBoostingClassifier: accuracy=0.8504, f1_score=0.9047
LogisticRegression: accuracy=0.8515, f1_score=0.9044
LogisticRegression: accuracy=0.8667, f1_score=0.9156
LogisticRegression: accuracy=0.8511, f1_score=0.9050
LogisticRegression: accuracy=0.8195, f1_score=0.8838
LogisticRegression: accuracy=0.8504, f1_score=0.9047
KNeighborsClassifier: accuracy=0.8515, f1_score=0.9044
KNeighborsClassifier: accuracy=0.8667, f1_score=0.9156
KNeighborsClassifier: accuracy=0.85

In [ ]:
results_classification